DETECTOR DE MATRICULAS FOTO

In [ ]:
import cv2
import math
import easyocr
from ultralytics import YOLO
import labelme

# Carga del modelo YOLO
model = YOLO('yolov8n.pt')

# Carga del modelo OCR para la detección de matrículas
reader = easyocr.Reader(['en'])

# Nombre de las distintas clases de YOLO
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

# Ruta de la imagen que deseas cargar
image_path = 'coche3.png'
img = cv2.imread(image_path)

# Crea un diccionario para almacenar las anotaciones de labelme
labelme_annotations = {"version": "4.5.7", "flags": {}, "shapes": [], "imagePath": image_path, "imageData": None}

# Realiza inferencia en la imagen con YOLO
results = model(img)

# Para cada detección
for r in results:
    boxes = r.boxes

    for box in boxes:
        # Contenedor
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convertir a valores enteros

        # Clase
        cls = int(box.cls[0])
        class_name = classNames[cls]

        # Solo continuamos si la clase es un vehículo
        if class_name in ["car", "motorbike", "bus"]:
            # Recorta la región de interés (ROI) que contiene el vehículo
            vehicle_roi = img[y1:y2, x1:x2]

            # Detección de matrícula con OCR
            results_ocr = reader.readtext(vehicle_roi)

            # Si se encuentra texto (matrícula)
            if results_ocr:
                letras = results_ocr[0][1]
                numeros = results_ocr[1][1]
                print(f"Matrícula del {class_name}: {numeros} {letras}")

            # Añade la anotación al diccionario labelme
            annotation = {"label": f'{class_name}: {numeros} {letras}' if results_ocr else class_name,
                          "points": [[x1, y1], [x2, y2]], "group_id": None, "shape_type": "rectangle", "flags": {}}
            labelme_annotations["shapes"].append(annotation)

            # Dibuja el contenedor y clase
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.putText(img, f'{class_name}: {numeros} {letras}' if results_ocr else class_name,
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Guarda el diccionario labelme en un archivo JSON
# labelme.save_json(image_path.replace('.png', '_labelme.json'), labelme_annotations)

# Muestra la imagen con las detecciones
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


DETECTOR DE MATRICULAS VIDEO

In [2]:
import cv2
import time
import easyocr
from ultralytics import YOLO

# Carga del modelo YOLO
model = YOLO('yolov8n.pt')

# Carga del modelo OCR para la detección de matrículas
reader = easyocr.Reader(['en'])

# Nombre de las distintas clases de YOLO
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]

# Ruta del video que deseas cargar
video_path = 'video2.mp4'
cap = cv2.VideoCapture(video_path)

# Establece un retraso de 100 milisegundos (0.1 segundos) entre fotogramas
delay = 100

while True:
    # Captura el fotograma del video
    ret, img = cap.read()

    if not ret:
        print("Fin del video.")
        break

    # Realiza inferencia en el fotograma con YOLO
    results = model(img)

    # Para cada detección
    for r in results:
        boxes = r.boxes

        for box in boxes:
            # Contenedor
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convertir a valores enteros

            # Clase
            cls = int(box.cls[0])
            class_name = classNames[cls]

            # Solo continuamos si la clase es un vehículo
            if class_name in ["car", "motorbike", "bus"]:
                # Recorta la región de interés (ROI) que contiene el vehículo
                vehicle_roi = img[y1:y2, x1:x2]

                # Detección de matrícula con OCR
                results_ocr = reader.readtext(vehicle_roi)

                # Si se encuentra texto (matrícula)
                if results_ocr:
                    #letras = results_ocr[0][1]
                    #numeros = results_ocr[1][1]
                    print(f"Matrícula del {class_name}: {results_ocr}")

            # Dibuja el contenedor y clase
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.putText(img, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Muestra el fotograma con las detecciones
    cv2.imshow('Video', img)

    # Introduce un retraso entre fotogramas
    time.sleep(delay / 1000.0)

    # Rompe el bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libera los recursos
cap.release()
cv2.destroyAllWindows()



0: 384x640 2 cars, 1 bus, 1 stop sign, 39.9ms
Speed: 1.4ms preprocess, 39.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 bus, 1 truck, 1 stop sign, 45.0ms
Speed: 1.7ms preprocess, 45.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 stop sign, 43.6ms
Speed: 1.6ms preprocess, 43.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 stop sign, 43.9ms
Speed: 1.6ms preprocess, 43.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 stop sign, 47.9ms
Speed: 1.5ms preprocess, 47.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 stop sign, 44.9ms
Speed: 1.6ms preprocess, 44.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 1 stop sign, 45.7ms
Speed: 1.5ms preprocess, 45.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars